In [0]:
%scala
import org.apache.spark.sql.types._
import java.sql.DriverManager
import scala.util.{Failure, Success, Try}

// Get params from Workflow
val sp_name = dbutils.widgets.get("sp_name")
val env = dbutils.widgets.get("environment")
val start_date = dbutils.widgets.get("start_date")
val end_date = dbutils.widgets.get("end_date")
val additional_query_info = s"@START_DATE = '$start_date', @END_DATE = '$end_date'"


// Connection parameters
val username = dbutils.secrets.get(scope = "state_reporting", key = s"sql_server_user_$env")
val password = dbutils.secrets.get(scope = "state_reporting", key = s"sql_server_pass_$env")
val host = dbutils.secrets.get(scope = "state_reporting", key = s"sql_server_host_$env")

val database = "statereporting"
val instaname = s"$env"

val sqlsUrl = s"jdbc:sqlserver://$host\\$instaname;databaseName=$database;encrypt=false"
val query = s"EXEC StateReporting.databricks.$sp_name $additional_query_info"


Try {
  // SQL query
  
  // Connection
  val connection = DriverManager.getConnection(sqlsUrl, username, password)
  try {
    val statement = connection.prepareCall(query)
    val resultSet = statement.executeQuery()
    while (resultSet.next()) {
      println(resultSet.getString(1)) // Example of reading results
    }
  } finally {
    // Ensure resources are closed
    connection.close()
  }
} match {
  case Success(_) => println("Stored Procedure executed successfully.")
  case Failure(exception) =>
    println(s"Error executing Stored Procedure: ${exception.getMessage}")
    throw new RuntimeException("Task failed due to an error while executing the stored procedure.", exception)
}

com.databricks.backend.daemon.driver.scalakernel.SparkException: No input widget named sp_name is defined
  com.databricks.backend.daemon.driver.scalakernel.extensions.CommUtils$.executeRequest(CommUtils.scala:56)
  com.databricks.backend.daemon.driver.scalakernel.extensions.Widgets.get(Widgets.scala:96)
  ammonite.$sess.cmd4$Helper.<init>(cmd4.sc:7)
  ammonite.$sess.cmd4$.<init>(cmd4.sc:7)
  ammonite.$sess.cmd4$.<clinit>(cmd4.sc:-1)